In [ ]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.manager.traffic_manager import PGTrafficManager
from metadrive.manager.pg_map_manager import PGMapManager
from metadrive.utils.config import Config
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert
from metadrive.component.map.base_map import BaseMap
from metadrive.component.map.pg_map import MapGenerateMethod
from metadrive.engine.logger import get_logger

import logging
import mediapy
from utils.scenario_runner import ScenarioRunner
from pathlib import Path
import numpy as np

log = get_logger()

In [ ]:
def get_default_config() -> dict:

    # ===== Termination Scheme =====
    termination_sceme = dict(
        out_of_route_done=False,
        on_continuous_line_done=False,
        crash_vehicle_done=False,
        crash_object_done=False,
        crash_human_done=False,
    )
    # ===== Map Config =====
    map_config = {
        BaseMap.GENERATE_TYPE: MapGenerateMethod.BIG_BLOCK_NUM,
        BaseMap.GENERATE_CONFIG: 3,  # 20 block
    }

    return dict(
        # use_render=True,
        log_level=logging.INFO,  # logging.DEBUG
        traffic_density=0.05,
        traffic_mode="respawn",
        random_traffic=False,
        decision_repeat=1,
        map_config=map_config,
        **termination_sceme,
    )

In [ ]:
class MultiFidelityTrafficManager(PGTrafficManager):
    def __init__(self):
        super().__init__()
        self.skip_counter = 0
        self.dr = self.global_config["decision_repeat"]
        self.dt = self.global_config["physics_world_step_size"]
        self.fps = 1 / (self.dr * self.dt)
        self.skip_limit = self.fps // 10

        log.info(
            f"Fidelity parameters in {self.__class__.__name__}: dr={self.dr}, dt={self.dt}, fps={self.fps}, skip={self.skip_limit}"
        )

    def before_step(self):
        """
        All traffic vehicles make driving decision here
        :return: None
        """
        # trigger vehicles
        assert self.mode == "respawn", "Currently only respawn is supported"

        log.info(
            f"Entering before step, skip_counter={self.skip_counter}, skip_limit={self.skip_limit}"
        )

        if self.skip_counter == 0:
            log.info(f"Setting BV policy")
            self.set_bv_policy()

        self.skip_counter += 1
        self.skip_counter %= self.skip_limit

        return dict()

    def set_bv_policy(self):
        for v in self._traffic_vehicles:
            p = self.engine.get_policy(v.name)
            v.before_step(p.act())

In [ ]:
class MultiFidelityEnv(MetaDriveEnv):

    def __init__(self, fps=10, seed=0, config=dict()):
        self.fps = fps
        config = config | {"start_seed": seed} | get_default_config()
        super(MultiFidelityEnv, self).__init__(config)

    def _post_process_config(self, config):
        config = super(MultiFidelityEnv, self)._post_process_config(config)
        config["physics_world_step_size"] = 1 / self.fps / config["decision_repeat"]

        return config

    def get_horizon(self):
        distance = self.agent.navigation.total_length
        V_min = 2.0  # [m/s]  # set minimal velocity to 2m/s
        max_time = distance / V_min  # [s] maximum time allowed to reach the destination
        max_steps = round(max_time * self.fps)  # maximum number of simulation steps frames
        return max_steps

    def setup_engine(self):
        super(MetaDriveEnv, self).setup_engine()
        self.engine.register_manager("map_manager", PGMapManager())
        self.engine.register_manager("traffic_manager", MultiFidelityTrafficManager())

In [ ]:
env = MultiFidelityEnv(fps=20, seed=10)
c = env.config
dt = env.config["physics_world_step_size"]
dr = c["decision_repeat"]
assert 1 / (dr * dt) == env.fps
log.info(f"Fidelity: {env.fps=} {dr=} {dt=:.2g}")

In [ ]:
_, reset_info = env.reset()

In [ ]:
action = expert(env.agent, deterministic=True)
obs, reward, terminated, truncated, info = env.step(action)

In [ ]:
env.close()

In [ ]:
def get_frames(fps):
    env = MultiFidelityEnv(fps, seed=1)
    _, reset_info = env.reset()
    frames = []
    for i in range(6000):
        action = expert(env.agent, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)

        b_box = env.current_map.road_network.get_bounding_box()
        x_len = b_box[1] - b_box[0]
        y_len = b_box[3] - b_box[2]
        scaling = 3
        frame = env.render(
            mode="topdown",
            window=False,
            # screen_size=(1080 // 2, 1920 // 2),
            screen_size=(x_len * scaling * 1.05, y_len * scaling * 1.05),
            camera_position=env.current_map.get_center_point(),
            scaling=scaling,
            draw_contour=True,
            num_stack=1,
            # screen_record=True,
        )
        frames.append(frame)
        if terminated or truncated:
            break
    env.close()
    return frames

In [ ]:
frames_a = get_frames(10)
frames_b = get_frames(20)

In [ ]:
mediapy.set_show_save_dir("data")

mediapy.show_videos([frames_a, frames_b[::2]], titles=["10FPS", "20FPS"], fps=10)
# mediapy.show_videos([frames_a, frames_b], titles=["First run", "Second run"], fps=10)

In [ ]:
env = MultiFidelityEnv(10)

In [ ]:
_ = env.reset()
action = expert(env.agent, deterministic=True)
obs, reward, terminated, truncated, info = env.step(action)

In [ ]:
# env.current_map.get_meta_data()
# env.engine.traffic_manager._traffic_vehicles